<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/master/experiments/1.1%5Bnext/Do(machineLearnings)/1126.MSE(image_conv)/training_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>